In [1]:
import pandas as pd 
import numpy as np
import os

In [2]:
#tuner_output_baseline = '/projects/home/wheyndri/research/wp1_imageauxdata/14_public_y3_imageaux_study/data/baseline/tuner_output/'
#tuner_input_baseline = '/projects/home/wheyndri/research/wp1_imageauxdata/14_public_y3_imageaux_study/data/image_model_input'

tuner_output_baseline = '/home/rama.jabal/Melloddy/aux_data/data/Mellody_tuner/all/output_all/pseudolabels_w_melloddy/'
tuner_input_images = '/home/rama.jabal/Melloddy/aux_data/data/Mellody_tuner/images/input_hofmarcher/'



# baseline tuner t8 file 
path_t8 = os.path.join(tuner_output_baseline, 'results_tmp/classification/T8c.csv')
t8 = pd.read_csv(path_t8)


# T0-file csv tuner input creation for the step1 image model
path_t0 = os.path.join(tuner_input_images,'T0.csv')
t0 = pd.read_csv(path_t0)

In [3]:
t8

,cont_classification_task_id,classification_task_id,input_assay_id,assay_type,variance_quorum_OK,is_auxiliary,use_in_regression,threshold,threshold_method,direction,training_quorum_OK,evaluation_quorum_OK,aggregation_weight,weight,num_total_actives,num_fold_min_actives,num_total_inactives,num_fold_min_inactives,n_tasks
0,NaN,1.0,517,OTHER,True,True,True,6.5,aux_low,high,False,False,0.0,NaN,52.0,3.0,19.0,1.0,1.0
1,0.0,2.0,517,OTHER,True,False,True,7.0,fixed_adaptive,high,True,False,0.0,1.0,37.0,2.0,34.0,2.0,1.0
2,NaN,3.0,517,OTHER,True,True,True,7.5,aux_high,high,False,False,0.0,NaN,23.0,1.0,48.0,3.0,1.0
3,1.0,4.0,1160,OTHER,True,True,True,6.5,aux_low,high,True,False,0.0,0.5,43.0,1.0,41.0,2.0,2.0
4,2.0,5.0,1160,OTHER,True,False,True,7.0,fixed_adaptive,high,True,False,0.0,0.5,30.0,1.0,54.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6580,NaN,NaN,4000015,ADME,False,NaN,False,NaN,NaN,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6581,NaN,NaN,4000017,ADME,False,NaN,False,NaN,NaN,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6582,NaN,NaN,4000022,ADME,False,NaN,False,NaN,NaN,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6583,NaN,NaN,4000035,ADME,False,NaN,False,NaN,NaN,high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
t8 = t8[['input_assay_id','threshold','threshold_method']]
t8 = t8.rename(columns={'threshold':'expert_threshold'})

In [5]:
t8['subgroup_index'] = t8.groupby('input_assay_id').cumcount()

In [6]:
df = pd.pivot_table(
    t8.drop_duplicates()
    ,columns=['subgroup_index']
    ,values=['expert_threshold']
    ,index=['input_assay_id']
)
df.columns = df.columns.map(lambda x: '_'.join([*map(str, x)]))


In [7]:
threshold_cols = [f'expert_threshold_{i}' for i in range(1,6)]
t0_out = pd.merge(
    t0.drop(columns=threshold_cols)
    ,df
    ,how='inner'
    ,on='input_assay_id'
)

In [8]:
# MT will freeze if not all the expert threshold columns are present
for col in threshold_cols :
    if col not in t0_out.columns : 
        t0_out[col] = np.nan

In [9]:
t0_out.sort_values(by='input_assay_id')

,input_assay_id,assay_type,use_in_regression,direction,expert_threshold_0,expert_threshold_1,expert_threshold_2,expert_threshold_3,expert_threshold_4,expert_threshold_5
303,1512,OTHER,True,NaN,7.500,8.000,8.500,NaN,NaN,NaN
255,1520,OTHER,True,NaN,7.500,8.000,8.500,NaN,NaN,NaN
155,2432,OTHER,True,NaN,6.855,7.355,7.855,NaN,NaN,NaN
443,3664,OTHER,True,NaN,7.500,8.000,8.500,NaN,NaN,NaN
481,4197,OTHER,True,NaN,5.500,6.000,6.500,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
755,4000031,ADME,False,high,0.000,NaN,NaN,NaN,NaN,NaN
821,4000032,ADME,False,high,0.000,NaN,NaN,NaN,NaN,NaN
822,4000033,ADME,False,high,0.000,NaN,NaN,NaN,NaN,NaN
823,4000034,ADME,False,high,0.000,NaN,NaN,NaN,NaN,NaN


In [10]:
t0_out.to_csv(os.path.join(tuner_input_images,'T0_synchedthreshold.csv'), index=False)

In [11]:
iais_in_scope = set(t0_out['input_assay_id'])

In [12]:
### Aligning also the T1 and T2 files

In [13]:
path_t1 = os.path.join(tuner_input_images,'T1.csv')
t1 = pd.read_csv(path_t1)

In [14]:
t1_synchedthreshold = t1.query('input_assay_id in @iais_in_scope')

In [15]:
t1_synchedthreshold.to_csv(os.path.join(tuner_input_images,'T1_synchedthreshold.csv'), index=False)

In [16]:
icis = set(t1_synchedthreshold['input_compound_id'])

In [17]:
path_t2 = os.path.join(tuner_input_images, 'T2.csv')
t2 = pd.read_csv(path_t2)

In [18]:
t2.query('input_compound_id in @icis').to_csv(os.path.join(tuner_input_images,'T2_synchedthreshold.csv'), index=False)